In [1]:
import pandas as pd
import numpy as np

test_df=pd.read_csv("test.csv")

categArr=test_df.select_dtypes(include="object").columns.to_list()
columns_to_remove_atAll = ["ID", "Customer_ID", "Name", "SSN","Month","Type_of_Loan","Credit_History_Age"]
columns_to_remove_transform=["Age" ,"Annual_Income" , "Num_of_Loan" ,"Num_of_Delayed_Payment" ,"Changed_Credit_Limit" ,"Outstanding_Debt" ,"Amount_invested_monthly" , "Monthly_Balance" ]

test_df=test_df.drop(columns=columns_to_remove_atAll)
for col in  columns_to_remove_transform :
  test_df[col]=pd.to_numeric(test_df[col], errors='coerce')

numeric_cols = test_df.select_dtypes(include=np.number).columns.tolist()
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy = 'mean')
imputer.fit(test_df[numeric_cols])
list(imputer.statistics_)
test_df[numeric_cols]=imputer.transform(test_df[numeric_cols])


categorical_cols = test_df.select_dtypes('object').columns.tolist()
test_df['Occupation'].replace("_______", np.nan, inplace=True)
test_df['Credit_Mix'].replace("_", np.nan, inplace=True)
test_df['Payment_of_Min_Amount'].replace("NM", np.nan, inplace=True)
test_df['Payment_Behaviour'].replace("!@9#%8", np.nan, inplace=True)
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(test_df[categorical_cols])
list(imputer.statistics_)
test_df[categorical_cols]=imputer.transform(test_df[categorical_cols])


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(test_df[numeric_cols])
list(scaler.data_min_)
list(scaler.data_max_)
test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])
test_df[numeric_cols].describe()

from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder.fit(test_df[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
test_df[encoded_cols] = encoder.transform(test_df[categorical_cols])






/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [2]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 46 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Age                                                 50000 non-null  float64
 1   Occupation                                          50000 non-null  object 
 2   Annual_Income                                       50000 non-null  float64
 3   Monthly_Inhand_Salary                               50000 non-null  float64
 4   Num_Bank_Accounts                                   50000 non-null  float64
 5   Num_Credit_Card                                     50000 non-null  float64
 6   Interest_Rate                                       50000 non-null  float64
 7   Num_of_Loan                                         50000 non-null  float64
 8   Delay_from_due_date                                 50000 non-null  float64


In [3]:
!pip install pyarrow --quiet
test_df.to_parquet('test_inputs.parquet')